고쳐할 내용은 인구수랑 기존의 데이터를 조인하면서 행정동이 더 작은 단위이긴 하지만 법정동이랑 미스매치되어서 그냥 뜨는 경우가 있음 그럴경우 중복되어서 나타나기 때문에 안예 그러한 값들을 제외 시켜주는 것이 낫다고 생각함. 
그리고 마지막에 최종적으로 인구수 컬럼이 null값이 없어야한다. Ex)세곡동을 참고해서 이해하기 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (7,153 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and d

## 런타임 재시작

In [4]:
%cd /content/drive/MyDrive/금융데이터 공모전

/content/drive/MyDrive/금융데이터 공모전


In [5]:

import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터전처리
### BC카드 데이터와 신한은행 데이터를 <성별, 연령, 법정동> 기준으로 조인하여 Sum한 값들을 인구수로 나누어주어서 1인당 값을 의미하는 데이터로 바꾸고자 한다. 

### 신한은행 데이터는 이미 개별로 수집된 데이터 이기 때문에 해당 값/수집된 값으로 계산할 것이며 BC카드는 서울시 인구수 데이터를 이용해서 값으로 나눌 것이다. 

EX) **BC카드** **데이터 : 디지털매출건수 / 특정조건의 서울시 인구수 >>> 조인은 <성별, 연령, 법정동>을 기준으로 조인 할 것이다.**

**신한은행 데이터 : 총수신금액 / 계산된 n값**


## 1. BC카드 데이터 1인당 값으로 전처리

### (1)인구수에 대한 데이터 전처리

In [7]:
refined_data= pd.read_csv("refined_data.csv").drop(columns='Unnamed: 0')
refined_data

,기준년월,법정동,성별,연령,DIY자재/용품_매출금액,DIY자재/용품_매출건수,e머니/상품권_매출금액,e머니/상품권_매출건수,o2o서비스_매출금액,o3o서비스_매출건수,가공식품_매출금액,가공식품_매출건수,가방_매출금액,가방_매출건수,가전_매출금액,가전_매출건수,건강식품_매출금액,건강식품_매출건수,기부/후원_매출금액,기부/후원_매출건수,기타_매출금액,기타_매출건수,기타결제_매출금액,기타결제_매출건수,기타교육비_매출금액,기타교육비_매출건수,남성뷰티_매출금액,남성뷰티_매출건수,남성속옷실내복_매출금액,남성속옷실내복_매출건수,남성의류_매출금액,남성의류_매출건수,다이어트_매출금액,다이어트_매출건수,디지털_매출금액,디지털_매출건수,레저_매출금액,레저_매출건수,메이크업_매출금액,메이크업_매출건수,...,여성속옷실내복_매출건수,여성의류_매출금액,여성의류_매출건수,여행_매출금액,여행_매출건수,육아용품서비스_매출금액,육아용품서비스_매출건수,음료_매출금액,음료_매출건수,인테리어소품_매출금액,인테리어소품_매출건수,임산부용품서비스_매출금액,임산부용품서비스_매출건수,자동차_매출금액,자동차_매출건수,취미/특기_매출금액,취미/특기_매출건수,침구/수예_매출금액,침구/수예_매출건수,침대_매출금액,침대_매출건수,테이블가구_매출금액,테이블가구_매출건수,패션소품_매출금액,패션소품_매출건수,향수_매출금액,향수_매출건수,급여입금,가맹점매출입금,연금입금,총소비금액,총수신금액,예적금금액,신탁금액,수익증권금액,신용대출금액,담보대출금액,주택대출금액,전세자금대출금액,n
0,1,가락동,1,20,0,0,104,702060.0,421,14482735,25,318720,0,0,0,0,42,862057,0,0.0,0,0,17,266816,0,0,0,0,0,0,25,1347488,False,False,67,5798971,0,0,0,0,...,0,0,0,75,2616631.0,0,0,0,0,0,0,0,0,0,0,104,4756975,0,0.0,0,0,0,0,0,0,0,0,6053415,0,0.0,276282702,637634013,9.622906e+08,0.0,0,0.0,0.000000e+00,0,0.0,345
1,1,가락동,1,30,21,1080521,196,3600015.0,1668,59595901,108,9026481,0,0,100,39755142,117,4135460,0,0.0,25,567484,42,1015110,0,0,0,0,0,0,21,957182,False,False,179,16190891,17,430116,0,0,...,0,25,830798,191,14826786.0,217,5987547,83,2626345,25,1594809,17,668324,29,471401,125,9718752,46,3075242.0,13,6226610,33,4203398,42,5478900,0,0,180731220,0,0.0,542263647,1733864271,1.801342e+09,27558987.0,0,412696176.0,2.196871e+09,0,0.0,411
2,1,가락동,1,40,21,697386,104,4246085.0,867,38478365,121,4147988,0,0,67,5106545,162,32823629,0,0.0,25,847366,63,506408,0,0,0,0,0,0,29,1517266,False,False,158,6323198,21,1207176,0,0,...,0,33,949035,105,9799656.0,87,2745620,96,3029070,25,4572509,21,576698,17,315802,84,12363378,63,2511968.0,0,0,0,0,29,1282589,0,0,122834583,0,0.0,499549200,1690930581,7.602494e+08,396429036.0,41064933,586295709.0,1.179034e+09,1469238294,0.0,369
3,1,가락동,1,50,17,2773427,58,2648983.0,221,10348725,29,2016253,0,0,25,5518572,38,3209088,0,0.0,0,0,442,2767466,0,0,0,0,0,0,17,968584,False,False,54,11018000,13,682882,0,0,...,0,0,0,29,3236812.0,25,359368,38,926352,0,0,0,0,0,0,92,6889960,17,1045485.0,0,0,0,0,0,0,0,0,60371844,6961017,0.0,546344004,1582574676,5.228890e+08,807396303.0,34321524,261313674.0,2.075667e+09,724597086,0.0,363
4,1,가락동,1,60,0,0,0,0.0,29,1317362,0,0,0,0,0,0,0,0,0,0.0,0,0,142,115481,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,...,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,46,10488912,0,0.0,0,0,0,0,0,0,0,0,9518316,13826772,65234532.0,586165779,2645038200,9.619000e+08,791406597.0,0,299537916.0,1.366055e+09,627738162,0.0,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14467,5,흥인동,1,50,0,0,0,0.0,0,0,0,0,0,0,0,0,5,157459,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,...,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,7565973,21540136,1.691960e+07,0.0,0,0.0,0.000000e+00,0,0.0,4
14468,5,흥인동,0,20,0,0,19,363764.0,179,5707264,14,269817,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,10,266084,0,0,0,0,...,334677,0,0,12,1026043.0,0,0,0,0,0,0,0,0,0,0,0,0,7,289857.0,0,0,0,0,5,432260,0,0,0,0,0.0,8887386,14462496,1.505395e+07,0.0,0,0.0,0.000000e+00,0,0.0,4
14469,5,흥인동,0,30,0,0,10,299624.0,201,8010103,17,657255,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,21,1278523,0,0,0,0,...,0,0,0,0,0.0,9,821315,9,329255,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,6467308,0,0.0,15377193,63203897,4.582949e+07,0.0,0,0.0,0.000000e+00,0,0.0,6
14470,5,흥인동,0,40,0,0,27,501504.0,120,7970038,12,353441,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,...,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,10,564135,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,9113575,29198407,1.760240e+07,4008189.0,1058035,17024229.0,1.328604e+08,0,0.0,4


In [8]:
#3개 데이터 모두 서울 뿐만 아니라 전국에 대한 수집 값
df1=pd.read_excel("법정동to행정동치환 자료.xlsx")
df2=pd.read_csv("인구수비중남자.csv", encoding="cp949")
df3=pd.read_csv('인구수비중여자.csv', encoding="cp949")

In [9]:
df1=df1[['행정동코드', '법정동코드', '법정동']]

In [10]:
df1

,행정동코드,법정동코드,법정동
0,11000000,11000000.0,서울특별시
1,11110000,11110000.0,종로구
2,11110515,11110101.0,청운동
3,11110515,11110102.0,신교동
4,11110515,11110103.0,궁정동
...,...,...,...
21734,50130610,50130114.0,대포동
21735,50130610,50130118.0,하원동
21736,50130620,50130119.0,색달동
21737,50130620,50130120.0,상예동


In [11]:
df2.head(30)

,행정동코드,성별,연령,인구가중치
0,11000000,1,20,1.926806
1,11000000,1,30,1.923680
2,11000000,1,40,1.921709
3,11000000,1,50,1.922575
4,11000000,1,60,1.894911
5,11110000,1,20,1.998810
6,11110000,1,30,1.998975
7,11110000,1,40,1.998879
8,11110000,1,50,1.998678
9,11110000,1,60,1.998155


In [12]:
df3

,행정동코드,성별,연령,인구수가중치
0,11000000,0,20,1.922145
1,11000000,0,30,1.923752
2,11000000,0,40,1.920232
3,11000000,0,50,1.919845
4,11000000,0,60,1.874843
...,...,...,...,...
19230,50130620,0,20,1.999975
19231,50130620,0,30,1.999980
19232,50130620,0,40,1.999975
19233,50130620,0,50,1.999968


In [13]:
join_popboy= pd.merge(left=df2, right=df1, how='left', on='행정동코드',left_index=True)
join_popboy.head(30)   ####예를 들어, 행정동 11110515 같은 경우는 >>다양한 법정동으로 치환이 가능해서 중복값으로 나오게 됨, 
                       #####하지만 우리는 결국 법정동으로 조인 할 꺼니깐 괜찮음>> left조인할때 필요한 것 만 갖가서 조인 하겠지

,행정동코드,성별,연령,인구가중치,법정동코드,법정동
0.0,11000000,1,20,1.926806,11000000.0,서울특별시
0.0,11000000,1,30,1.923680,11000000.0,서울특별시
0.0,11000000,1,40,1.921709,11000000.0,서울특별시
0.0,11000000,1,50,1.922575,11000000.0,서울특별시
0.0,11000000,1,60,1.894911,11000000.0,서울특별시
1.0,11110000,1,20,1.998810,11110000.0,종로구
1.0,11110000,1,30,1.998975,11110000.0,종로구
1.0,11110000,1,40,1.998879,11110000.0,종로구
1.0,11110000,1,50,1.998678,11110000.0,종로구
1.0,11110000,1,60,1.998155,11110000.0,종로구


In [14]:
### 중복된 값들이 많다는 것을 알 수 있음
join_popboy[join_popboy.duplicated(['행정동코드',	'성별',	'연령',	'인구가중치'])]

,행정동코드,성별,연령,인구가중치,법정동코드,법정동
3.0,11110515,1,20,1.999918,11110102.0,신교동
4.0,11110515,1,20,1.999918,11110103.0,궁정동
5.0,11110515,1,20,1.999918,11110104.0,효자동
6.0,11110515,1,20,1.999918,11110105.0,창성동
7.0,11110515,1,20,1.999918,11110108.0,통인동
...,...,...,...,...,...,...
21738.0,50130620,1,40,1.999969,50130121.0,하예동
21737.0,50130620,1,50,1.999960,50130120.0,상예동
21738.0,50130620,1,50,1.999960,50130121.0,하예동
21737.0,50130620,1,60,1.999941,50130120.0,상예동


In [15]:
##남자 인구 통계 데이터 프레임
join_popboy=join_popboy[['법정동코드','법정동','성별','연령','인구가중치']]
join_popboy['인구가중치']= (2-join_popboy['인구가중치'])*9668465
join_popboy.rename(columns={'인구가중치':'인구수'}, inplace=True)
join_popboy

,법정동코드,법정동,성별,연령,인구수
0.0,11000000.0,서울특별시,1,20,7.076690e+05
0.0,11000000.0,서울특별시,1,30,7.379010e+05
0.0,11000000.0,서울특별시,1,40,7.569500e+05
0.0,11000000.0,서울특별시,1,50,7.485820e+05
0.0,11000000.0,서울특별시,1,60,1.016049e+06
...,...,...,...,...,...
21737.0,50130120.0,상예동,1,50,3.829679e+02
21738.0,50130121.0,하예동,1,50,3.829679e+02
21736.0,50130119.0,색달동,1,60,5.749836e+02
21737.0,50130120.0,상예동,1,60,5.749836e+02


In [16]:
##남자 인구 통계 데이터 프레임
join_popgirl= pd.merge(left=df3, right=df1, how='left', on='행정동코드',left_index=True)
join_popgirl=join_popgirl[['법정동코드','법정동','성별','연령','인구수가중치']]
join_popgirl['인구수가중치']= (2-join_popgirl['인구수가중치'])*9668465
join_popgirl.rename(columns={'인구수가중치':'인구수'}, inplace=True)
join_popgirl

,법정동코드,법정동,성별,연령,인구수
0.0,11000000.0,서울특별시,0,20,7.527420e+05
0.0,11000000.0,서울특별시,0,30,7.372050e+05
0.0,11000000.0,서울특별시,0,40,7.712320e+05
0.0,11000000.0,서울특별시,0,50,7.749740e+05
0.0,11000000.0,서울특별시,0,60,1.210075e+06
...,...,...,...,...,...
21737.0,50130120.0,상예동,0,50,3.060069e+02
21738.0,50130121.0,하예동,0,50,3.060069e+02
21736.0,50130119.0,색달동,0,60,6.389985e+02
21737.0,50130120.0,상예동,0,60,6.389985e+02


In [17]:
merge_popul=pd.concat([join_popboy,join_popgirl])
# merge_popul.to_csv('merge_popul.csv')
merge_popul

,법정동코드,법정동,성별,연령,인구수
0.0,11000000.0,서울특별시,1,20,7.076690e+05
0.0,11000000.0,서울특별시,1,30,7.379010e+05
0.0,11000000.0,서울특별시,1,40,7.569500e+05
0.0,11000000.0,서울특별시,1,50,7.485820e+05
0.0,11000000.0,서울특별시,1,60,1.016049e+06
...,...,...,...,...,...
21737.0,50130120.0,상예동,0,50,3.060069e+02
21738.0,50130121.0,하예동,0,50,3.060069e+02
21736.0,50130119.0,색달동,0,60,6.389985e+02
21737.0,50130120.0,상예동,0,60,6.389985e+02


### (2)BC카드 1인당 값으로 나누기

In [18]:
df=pd.read_excel('머신러닝을 위한 학습자료(월을 기준으로 합쳐서 숫자화).xlsx')
df1=pd.read_csv('BC card.csv')
df

,기준년월,품목대분류코드,품목중분류코드,성별,연령,가구생애주기,시군구,법정동코드,매출금액,매출건수,예금금액,소비금액,주택대출금액,전세자금대출,인구밀도
0,201903,10,1010,0,40.0,3,구로구,11530109.0,782521,13,NaN,NaN,NaN,NaN,NaN
1,201903,21,2110,0,20.0,2,구로구,11530109.0,963789,29,NaN,NaN,NaN,NaN,NaN
2,201903,19,1913,0,30.0,2,구로구,11530109.0,924309,46,NaN,NaN,NaN,NaN,NaN
3,201903,21,2111,0,30.0,2,구로구,11530109.0,1089360,54,NaN,NaN,NaN,NaN,NaN
4,201903,17,1710,0,30.0,2,구로구,11530109.0,1656740,58,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306117,202103,20,2013,0,30.0,1,영등포구,11560108.0,1411368,7,NaN,NaN,NaN,NaN,NaN
306118,202103,21,2110,1,30.0,1,영등포구,11560108.0,1386040,63,NaN,NaN,NaN,NaN,NaN
306119,202103,11,1113,0,30.0,2,영등포구,11560108.0,390818,7,NaN,NaN,NaN,NaN,NaN
306120,202103,16,1610,1,40.0,3,영등포구,11560108.0,327592,7,NaN,NaN,NaN,NaN,NaN


In [19]:
df1['법정동코드']=df['법정동코드']
df2=df1[['읍면동','법정동코드']]

In [20]:
df2[df2.duplicated(['읍면동','법정동코드'])]

,읍면동,법정동코드
1,궁동,11530109.0
2,궁동,11530109.0
3,궁동,11530109.0
4,궁동,11530109.0
6,길동,11740105.0
...,...,...
306117,영등포동7가,11560108.0
306118,영등포동7가,11560108.0
306119,영등포동7가,11560108.0
306120,영등포동7가,11560108.0


In [39]:
df2=df2.drop_duplicates(subset=['읍면동', '법정동코드'], keep='first)

In [22]:
df2[df2['법정동코드'].isnull()]  ## 궁정동: 11110103 도럼동:11110116 봉래동2가:11140120

,읍면동,법정동코드
75512,궁정동,NaN
135887,도렴동,NaN
260662,봉래동2가,NaN


In [23]:
df2.loc[75512,'법정동코드']=11110103
df2.loc[135887,'법정동코드']=11110116
df2.loc[260662,'법정동코드']=111140120

In [24]:
df2.isnull().sum()

읍면동      0
법정동코드    0
dtype: int64

In [25]:
refined_data_final=pd.merge(left=refined_data, right=df2, how='left', left_on='법정동', right_on='읍면동')
refined_data_final=refined_data_final.drop(columns='읍면동')
refined_data_final['법정동코드'].isnull().sum() ####결측치가 없음을 확인

0

In [26]:
col1=refined_data_final.columns[1:].to_list()
col2=refined_data_final.columns[:1].to_list()
new_col = col1+col2
refined_data_final = refined_data_final[new_col]

In [27]:
refined_data_final

,법정동,성별,연령,DIY자재/용품_매출금액,DIY자재/용품_매출건수,e머니/상품권_매출금액,e머니/상품권_매출건수,o2o서비스_매출금액,o3o서비스_매출건수,가공식품_매출금액,가공식품_매출건수,가방_매출금액,가방_매출건수,가전_매출금액,가전_매출건수,건강식품_매출금액,건강식품_매출건수,기부/후원_매출금액,기부/후원_매출건수,기타_매출금액,기타_매출건수,기타결제_매출금액,기타결제_매출건수,기타교육비_매출금액,기타교육비_매출건수,남성뷰티_매출금액,남성뷰티_매출건수,남성속옷실내복_매출금액,남성속옷실내복_매출건수,남성의류_매출금액,남성의류_매출건수,다이어트_매출금액,다이어트_매출건수,디지털_매출금액,디지털_매출건수,레저_매출금액,레저_매출건수,메이크업_매출금액,메이크업_매출건수,모바일상품_매출금액,...,여성의류_매출건수,여행_매출금액,여행_매출건수,육아용품서비스_매출금액,육아용품서비스_매출건수,음료_매출금액,음료_매출건수,인테리어소품_매출금액,인테리어소품_매출건수,임산부용품서비스_매출금액,임산부용품서비스_매출건수,자동차_매출금액,자동차_매출건수,취미/특기_매출금액,취미/특기_매출건수,침구/수예_매출금액,침구/수예_매출건수,침대_매출금액,침대_매출건수,테이블가구_매출금액,테이블가구_매출건수,패션소품_매출금액,패션소품_매출건수,향수_매출금액,향수_매출건수,급여입금,가맹점매출입금,연금입금,총소비금액,총수신금액,예적금금액,신탁금액,수익증권금액,신용대출금액,담보대출금액,주택대출금액,전세자금대출금액,n,법정동코드,기준년월
0,가락동,1,20,0,0,104,702060.0,421,14482735,25,318720,0,0,0,0,42,862057,0,0.0,0,0,17,266816,0,0,0,0,0,0,25,1347488,False,False,67,5798971,0,0,0,0,13,...,0,75,2616631.0,0,0,0,0,0,0,0,0,0,0,104,4756975,0,0.0,0,0,0,0,0,0,0,0,6053415,0,0.0,276282702,637634013,9.622906e+08,0.0,0,0.0,0.000000e+00,0,0.0,345,11710107.0,1
1,가락동,1,30,21,1080521,196,3600015.0,1668,59595901,108,9026481,0,0,100,39755142,117,4135460,0,0.0,25,567484,42,1015110,0,0,0,0,0,0,21,957182,False,False,179,16190891,17,430116,0,0,38,...,830798,191,14826786.0,217,5987547,83,2626345,25,1594809,17,668324,29,471401,125,9718752,46,3075242.0,13,6226610,33,4203398,42,5478900,0,0,180731220,0,0.0,542263647,1733864271,1.801342e+09,27558987.0,0,412696176.0,2.196871e+09,0,0.0,411,11710107.0,1
2,가락동,1,40,21,697386,104,4246085.0,867,38478365,121,4147988,0,0,67,5106545,162,32823629,0,0.0,25,847366,63,506408,0,0,0,0,0,0,29,1517266,False,False,158,6323198,21,1207176,0,0,38,...,949035,105,9799656.0,87,2745620,96,3029070,25,4572509,21,576698,17,315802,84,12363378,63,2511968.0,0,0,0,0,29,1282589,0,0,122834583,0,0.0,499549200,1690930581,7.602494e+08,396429036.0,41064933,586295709.0,1.179034e+09,1469238294,0.0,369,11710107.0,1
3,가락동,1,50,17,2773427,58,2648983.0,221,10348725,29,2016253,0,0,25,5518572,38,3209088,0,0.0,0,0,442,2767466,0,0,0,0,0,0,17,968584,False,False,54,11018000,13,682882,0,0,0,...,0,29,3236812.0,25,359368,38,926352,0,0,0,0,0,0,92,6889960,17,1045485.0,0,0,0,0,0,0,0,0,60371844,6961017,0.0,546344004,1582574676,5.228890e+08,807396303.0,34321524,261313674.0,2.075667e+09,724597086,0.0,363,11710107.0,1
4,가락동,1,60,0,0,0,0.0,29,1317362,0,0,0,0,0,0,0,0,0,0.0,0,0,142,115481,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0,0,0,0,46,10488912,0,0.0,0,0,0,0,0,0,0,0,9518316,13826772,65234532.0,586165779,2645038200,9.619000e+08,791406597.0,0,299537916.0,1.366055e+09,627738162,0.0,582,11710107.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14467,흥인동,1,50,0,0,0,0.0,0,0,0,0,0,0,0,0,5,157459,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,7565973,21540136,1.691960e+07,0.0,0,0.0,0.000000e+00,0,0.0,4,11140163.0,5
14468,흥인동,0,20,0,0,19,363764.0,179,5707264,14,269817,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,10,266084,0,0,0,0,0,...,0,12,1026043.0,0,0,0,0,0,0,0,0,0,0,0,0,7,289857.0,0,0,0,0,5,432260,0,0,0,0,0.0,8887386,14462496,1.505395e+07,0.0,0,0.0,0.000000e+00,0,0.0,4,11140163.0,5
14469,흥인동,0,30,0,0,10,299624.0,201,8010103,17,657255,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,21,1278523,0,0,0,0,0,...,0,0,0.0,9,821315,9,329255,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,6467308,0,0.0,15377193,63203897,4.582949e+07,0.0,0,0.0,0.000000e+00,0,0.0,6,11140163.0,5
14470,흥인동,0,40,0,0,27,501504.0,120,7970038,12,353441,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0,0,0,0,10,564135,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,9113575,29198407,1.760240e+07,4008189.0,

In [28]:
##merge_popul에서 법정동을 drop해보자
merge_popul=merge_popul.drop(columns='법정동')

In [29]:
merge_popul

,법정동코드,성별,연령,인구수
0.0,11000000.0,1,20,7.076690e+05
0.0,11000000.0,1,30,7.379010e+05
0.0,11000000.0,1,40,7.569500e+05
0.0,11000000.0,1,50,7.485820e+05
0.0,11000000.0,1,60,1.016049e+06
...,...,...,...,...
21737.0,50130120.0,0,50,3.060069e+02
21738.0,50130121.0,0,50,3.060069e+02
21736.0,50130119.0,0,60,6.389985e+02
21737.0,50130120.0,0,60,6.389985e+02


In [30]:
###중복값 제거를 위한 sum 작업
merge_popul2=merge_popul['인구수'].groupby([merge_popul['법정동코드'], merge_popul['성별'],merge_popul['연령']]).sum()
merge_popul2=pd.DataFrame(merge_popul2)
merge_popul2.to_csv("merge_popul2.csv")
merge_popul2=pd.read_csv("merge_popul2.csv")
merge_popul2

,법정동코드,성별,연령,인구수
0,11000000.00,0,20,7.527420e+05
1,11000000.00,0,30,7.372050e+05
2,11000000.00,0,40,7.712320e+05
3,11000000.00,0,50,7.749740e+05
4,11000000.00,0,60,1.210075e+06
...,...,...,...,...
204165,50130320.26,1,20,6.960328e+02
204166,50130320.26,1,30,5.429810e+02
204167,50130320.26,1,40,1.114001e+03
204168,50130320.26,1,50,1.274980e+03


In [31]:
merge_popul=merge_popul2

정제된 데이터로 조인 실시

In [32]:
###left로 조인해서 몇개의 데이터의 손실이 있지만 괜찮다고 생각
refined_Join=pd.merge(left=refined_data_final, right=merge_popul, how='left', on=['법정동코드','성별','연령'])
refined_Join

,법정동,성별,연령,DIY자재/용품_매출금액,DIY자재/용품_매출건수,e머니/상품권_매출금액,e머니/상품권_매출건수,o2o서비스_매출금액,o3o서비스_매출건수,가공식품_매출금액,가공식품_매출건수,가방_매출금액,가방_매출건수,가전_매출금액,가전_매출건수,건강식품_매출금액,건강식품_매출건수,기부/후원_매출금액,기부/후원_매출건수,기타_매출금액,기타_매출건수,기타결제_매출금액,기타결제_매출건수,기타교육비_매출금액,기타교육비_매출건수,남성뷰티_매출금액,남성뷰티_매출건수,남성속옷실내복_매출금액,남성속옷실내복_매출건수,남성의류_매출금액,남성의류_매출건수,다이어트_매출금액,다이어트_매출건수,디지털_매출금액,디지털_매출건수,레저_매출금액,레저_매출건수,메이크업_매출금액,메이크업_매출건수,모바일상품_매출금액,...,여행_매출금액,여행_매출건수,육아용품서비스_매출금액,육아용품서비스_매출건수,음료_매출금액,음료_매출건수,인테리어소품_매출금액,인테리어소품_매출건수,임산부용품서비스_매출금액,임산부용품서비스_매출건수,자동차_매출금액,자동차_매출건수,취미/특기_매출금액,취미/특기_매출건수,침구/수예_매출금액,침구/수예_매출건수,침대_매출금액,침대_매출건수,테이블가구_매출금액,테이블가구_매출건수,패션소품_매출금액,패션소품_매출건수,향수_매출금액,향수_매출건수,급여입금,가맹점매출입금,연금입금,총소비금액,총수신금액,예적금금액,신탁금액,수익증권금액,신용대출금액,담보대출금액,주택대출금액,전세자금대출금액,n,법정동코드,기준년월,인구수
0,가락동,1,20,0,0,104,702060.0,421,14482735,25,318720,0,0,0,0,42,862057,0,0.0,0,0,17,266816,0,0,0,0,0,0,25,1347488,False,False,67,5798971,0,0,0,0,13,...,75,2616631.0,0,0,0,0,0,0,0,0,0,0,104,4756975,0,0.0,0,0,0,0,0,0,0,0,6053415,0,0.0,276282702,637634013,9.622906e+08,0.0,0,0.0,0.000000e+00,0,0.0,345,11710107.0,1,6113.950527
1,가락동,1,30,21,1080521,196,3600015.0,1668,59595901,108,9026481,0,0,100,39755142,117,4135460,0,0.0,25,567484,42,1015110,0,0,0,0,0,0,21,957182,False,False,179,16190891,17,430116,0,0,38,...,191,14826786.0,217,5987547,83,2626345,25,1594809,17,668324,29,471401,125,9718752,46,3075242.0,13,6226610,33,4203398,42,5478900,0,0,180731220,0,0.0,542263647,1733864271,1.801342e+09,27558987.0,0,412696176.0,2.196871e+09,0,0.0,411,11710107.0,1,7302.978353
2,가락동,1,40,21,697386,104,4246085.0,867,38478365,121,4147988,0,0,67,5106545,162,32823629,0,0.0,25,847366,63,506408,0,0,0,0,0,0,29,1517266,False,False,158,6323198,21,1207176,0,0,38,...,105,9799656.0,87,2745620,96,3029070,25,4572509,21,576698,17,315802,84,12363378,63,2511968.0,0,0,0,0,29,1282589,0,0,122834583,0,0.0,499549200,1690930581,7.602494e+08,396429036.0,41064933,586295709.0,1.179034e+09,1469238294,0.0,369,11710107.0,1,6806.019252
3,가락동,1,50,17,2773427,58,2648983.0,221,10348725,29,2016253,0,0,25,5518572,38,3209088,0,0.0,0,0,442,2767466,0,0,0,0,0,0,17,968584,False,False,54,11018000,13,682882,0,0,0,...,29,3236812.0,25,359368,38,926352,0,0,0,0,0,0,92,6889960,17,1045485.0,0,0,0,0,0,0,0,0,60371844,6961017,0.0,546344004,1582574676,5.228890e+08,807396303.0,34321524,261313674.0,2.075667e+09,724597086,0.0,363,11710107.0,1,6766.088492
4,가락동,1,60,0,0,0,0.0,29,1317362,0,0,0,0,0,0,0,0,0,0.0,0,0,142,115481,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0,0,0,46,10488912,0,0.0,0,0,0,0,0,0,0,0,9518316,13826772,65234532.0,586165779,2645038200,9.619000e+08,791406597.0,0,299537916.0,1.366055e+09,627738162,0.0,582,11710107.0,1,8203.982606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14467,흥인동,1,50,0,0,0,0.0,0,0,0,0,0,0,0,0,5,157459,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,7565973,21540136,1.691960e+07,0.0,0,0.0,0.000000e+00,0,0.0,4,11140163.0,5,743.988382
14468,흥인동,0,20,0,0,19,363764.0,179,5707264,14,269817,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,10,266084,0,0,0,0,0,...,12,1026043.0,0,0,0,0,0,0,0,0,0,0,0,0,7,289857.0,0,0,0,0,5,432260,0,0,0,0,0.0,8887386,14462496,1.505395e+07,0.0,0,0.0,0.000000e+00,0,0.0,4,11140163.0,5,714.982987
14469,흥인동,0,30,0,0,10,299624.0,201,8010103,17,657255,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,21,1278523,0,0,0,0,0,...,0,0.0,9,821315,9,329255,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,6467308,0,0.0,15377193,63203897,4.582949e+07,0.0,0,0.0,0.000000e+00,0,0.0,6,11140163.0,5,641.986076
14470,흥인동,0,40,0,0,27,501504.0,120,7970038,12,353441,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0,0,0,10,564135,0,0.0,0,0,0

In [41]:
refined_Join['인구수'].isnull().sum()

50

BC카드 데이터만 먼저 1인당 데이터화 시켜줌

In [33]:
list_col=refined_Join.columns[2:102].to_list()

In [34]:
for i in list_col:
  refined_Join[i]=refined_Join[i]/refined_Join['인구수']
refined_Join

,법정동,성별,연령,DIY자재/용품_매출금액,DIY자재/용품_매출건수,e머니/상품권_매출금액,e머니/상품권_매출건수,o2o서비스_매출금액,o3o서비스_매출건수,가공식품_매출금액,가공식품_매출건수,가방_매출금액,가방_매출건수,가전_매출금액,가전_매출건수,건강식품_매출금액,건강식품_매출건수,기부/후원_매출금액,기부/후원_매출건수,기타_매출금액,기타_매출건수,기타결제_매출금액,기타결제_매출건수,기타교육비_매출금액,기타교육비_매출건수,남성뷰티_매출금액,남성뷰티_매출건수,남성속옷실내복_매출금액,남성속옷실내복_매출건수,남성의류_매출금액,남성의류_매출건수,다이어트_매출금액,다이어트_매출건수,디지털_매출금액,디지털_매출건수,레저_매출금액,레저_매출건수,메이크업_매출금액,메이크업_매출건수,모바일상품_매출금액,...,여행_매출금액,여행_매출건수,육아용품서비스_매출금액,육아용품서비스_매출건수,음료_매출금액,음료_매출건수,인테리어소품_매출금액,인테리어소품_매출건수,임산부용품서비스_매출금액,임산부용품서비스_매출건수,자동차_매출금액,자동차_매출건수,취미/특기_매출금액,취미/특기_매출건수,침구/수예_매출금액,침구/수예_매출건수,침대_매출금액,침대_매출건수,테이블가구_매출금액,테이블가구_매출건수,패션소품_매출금액,패션소품_매출건수,향수_매출금액,향수_매출건수,급여입금,가맹점매출입금,연금입금,총소비금액,총수신금액,예적금금액,신탁금액,수익증권금액,신용대출금액,담보대출금액,주택대출금액,전세자금대출금액,n,법정동코드,기준년월,인구수
0,가락동,1,0.003271,0.000000,0.000000,0.017010,114.829192,0.068859,2368.801471,0.004089,52.129961,0.0,0.0,0.000000,0.000000,0.006870,140.998360,0.0,0.0,0.000000,0.000000,0.002781,43.640523,0.0,0.0,0.0,0.0,0.0,0.0,0.004089,220.395634,0.0,0.0,0.010959,948.481832,0.000000,0.000000,0.0,0.0,0.002126,...,0.012267,427.977130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017010,778.052583,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,6053415,0,0.0,276282702,637634013,9.622906e+08,0.0,0,0.0,0.000000e+00,0,0.0,345,11710107.0,1,6113.950527
1,가락동,1,0.004108,0.002876,147.956210,0.026838,492.951619,0.228400,8160.492626,0.014788,1235.999967,0.0,0.0,0.013693,5443.688873,0.016021,566.270335,0.0,0.0,0.003423,77.705831,0.005751,138.999453,0.0,0.0,0.0,0.0,0.0,0.0,0.002876,131.067347,0.0,0.0,0.024511,2217.025742,0.002328,58.895971,0.0,0.0,0.005203,...,0.026154,2030.238251,0.029714,819.877413,0.011365,359.626562,0.003423,218.377890,0.002328,91.513896,0.003971,64.549144,0.017116,1330.792935,0.006299,421.094224,0.00178,852.612414,0.004519,575.573115,0.005751,750.228158,0.0,0,180731220,0,0.0,542263647,1733864271,1.801342e+09,27558987.0,0,412696176.0,2.196871e+09,0,0.0,411,11710107.0,1,7302.978353
2,가락동,1,0.005877,0.003086,102.466063,0.015281,623.872023,0.127387,5653.578630,0.017778,609.458752,0.0,0.0,0.009844,750.298348,0.023802,4822.735256,0.0,0.0,0.003673,124.502439,0.009257,74.405902,0.0,0.0,0.0,0.0,0.0,0.0,0.004261,222.930019,0.0,0.0,0.023215,929.059670,0.003086,177.368878,0.0,0.0,0.005583,...,0.015428,1439.851349,0.012783,403.410554,0.014105,445.057513,0.003673,671.833098,0.003086,84.733525,0.002498,46.400398,0.012342,1816.535855,0.009257,369.080355,0.00000,0.000000,0.000000,0.000000,0.004261,188.449217,0.0,0,122834583,0,0.0,499549200,1690930581,7.602494e+08,396429036.0,41064933,586295709.0,1.179034e+09,1469238294,0.0,369,11710107.0,1,6806.019252
3,가락동,1,0.007390,0.002513,409.901083,0.008572,391.508773,0.032663,1529.498914,0.004286,297.993886,0.0,0.0,0.003695,815.622203,0.005616,474.289984,0.0,0.0,0.000000,0.000000,0.065326,409.020072,0.0,0.0,0.0,0.0,0.0,0.0,0.002513,143.152724,0.0,0.0,0.007981,1628.415001,0.001921,100.927146,0.0,0.0,0.000000,...,0.004286,478.387477,0.003695,53.113110,0.005616,136.911009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013597,1018.307699,0.002513,154.518375,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,60371844,6961017,0.0,546344004,1582574676,5.228890e+08,807396303.0,34321524,261313674.0,2.075667e+09,724597086,0.0,363,11710107.0,1,6766.088492
4,가락동,1,0.007314,0.000000,0.000000,0.000000,0.000000,0.003535,160.575913,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.017309,14.076212,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005607,1278.514656,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,9518316,13826772,65234532.0,586165779,2645038200,9.619000e+08,791406597.0,0,299537916.0,1.366055e+09,627738162,0.0,582,11710107.0,1,8203.982606
...,...,...,...,...,...,...,..

지금부터는 신한은행데이터를 1인당 데이터화 시켜줌

In [35]:
list_col2=refined_Join.columns[103:114].to_list()
list_col2

['급여입금',
 '가맹점매출입금',
 '연금입금',
 '총소비금액',
 '총수신금액',
 '예적금금액',
 '신탁금액',
 '수익증권금액',
 '신용대출금액',
 '담보대출금액',
 '주택대출금액']

In [36]:
for i in list_col2:
  refined_Join[i]=refined_Join[i]/refined_Join['n']
refined_Join

,법정동,성별,연령,DIY자재/용품_매출금액,DIY자재/용품_매출건수,e머니/상품권_매출금액,e머니/상품권_매출건수,o2o서비스_매출금액,o3o서비스_매출건수,가공식품_매출금액,가공식품_매출건수,가방_매출금액,가방_매출건수,가전_매출금액,가전_매출건수,건강식품_매출금액,건강식품_매출건수,기부/후원_매출금액,기부/후원_매출건수,기타_매출금액,기타_매출건수,기타결제_매출금액,기타결제_매출건수,기타교육비_매출금액,기타교육비_매출건수,남성뷰티_매출금액,남성뷰티_매출건수,남성속옷실내복_매출금액,남성속옷실내복_매출건수,남성의류_매출금액,남성의류_매출건수,다이어트_매출금액,다이어트_매출건수,디지털_매출금액,디지털_매출건수,레저_매출금액,레저_매출건수,메이크업_매출금액,메이크업_매출건수,모바일상품_매출금액,...,여행_매출금액,여행_매출건수,육아용품서비스_매출금액,육아용품서비스_매출건수,음료_매출금액,음료_매출건수,인테리어소품_매출금액,인테리어소품_매출건수,임산부용품서비스_매출금액,임산부용품서비스_매출건수,자동차_매출금액,자동차_매출건수,취미/특기_매출금액,취미/특기_매출건수,침구/수예_매출금액,침구/수예_매출건수,침대_매출금액,침대_매출건수,테이블가구_매출금액,테이블가구_매출건수,패션소품_매출금액,패션소품_매출건수,향수_매출금액,향수_매출건수,급여입금,가맹점매출입금,연금입금,총소비금액,총수신금액,예적금금액,신탁금액,수익증권금액,신용대출금액,담보대출금액,주택대출금액,전세자금대출금액,n,법정동코드,기준년월,인구수
0,가락동,1,0.003271,0.000000,0.000000,0.017010,114.829192,0.068859,2368.801471,0.004089,52.129961,0.0,0.0,0.000000,0.000000,0.006870,140.998360,0.0,0.0,0.000000,0.000000,0.002781,43.640523,0.0,0.0,0.0,0.0,0.0,0.0,0.004089,220.395634,0.0,0.0,0.010959,948.481832,0.000000,0.000000,0.0,0.0,0.002126,...,0.012267,427.977130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017010,778.052583,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,1.754613e+04,0.000000,0.000000,8.008194e+05,1.848215e+06,2.789248e+06,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,345,11710107.0,1,6113.950527
1,가락동,1,0.004108,0.002876,147.956210,0.026838,492.951619,0.228400,8160.492626,0.014788,1235.999967,0.0,0.0,0.013693,5443.688873,0.016021,566.270335,0.0,0.0,0.003423,77.705831,0.005751,138.999453,0.0,0.0,0.0,0.0,0.0,0.0,0.002876,131.067347,0.0,0.0,0.024511,2217.025742,0.002328,58.895971,0.0,0.0,0.005203,...,0.026154,2030.238251,0.029714,819.877413,0.011365,359.626562,0.003423,218.377890,0.002328,91.513896,0.003971,64.549144,0.017116,1330.792935,0.006299,421.094224,0.00178,852.612414,0.004519,575.573115,0.005751,750.228158,0.0,0,4.397353e+05,0.000000,0.000000,1.319376e+06,4.218648e+06,4.382826e+06,6.705350e+04,0.000000,1.004127e+06,5.345186e+06,0.000000e+00,0.0,411,11710107.0,1,7302.978353
2,가락동,1,0.005877,0.003086,102.466063,0.015281,623.872023,0.127387,5653.578630,0.017778,609.458752,0.0,0.0,0.009844,750.298348,0.023802,4822.735256,0.0,0.0,0.003673,124.502439,0.009257,74.405902,0.0,0.0,0.0,0.0,0.0,0.0,0.004261,222.930019,0.0,0.0,0.023215,929.059670,0.003086,177.368878,0.0,0.0,0.005583,...,0.015428,1439.851349,0.012783,403.410554,0.014105,445.057513,0.003673,671.833098,0.003086,84.733525,0.002498,46.400398,0.012342,1816.535855,0.009257,369.080355,0.00000,0.000000,0.000000,0.000000,0.004261,188.449217,0.0,0,3.328850e+05,0.000000,0.000000,1.353792e+06,4.582468e+06,2.060297e+06,1.074333e+06,111287.081301,1.588877e+06,3.195215e+06,3.981676e+06,0.0,369,11710107.0,1,6806.019252
3,가락동,1,0.007390,0.002513,409.901083,0.008572,391.508773,0.032663,1529.498914,0.004286,297.993886,0.0,0.0,0.003695,815.622203,0.005616,474.289984,0.0,0.0,0.000000,0.000000,0.065326,409.020072,0.0,0.0,0.0,0.0,0.0,0.0,0.002513,143.152724,0.0,0.0,0.007981,1628.415001,0.001921,100.927146,0.0,0.0,0.000000,...,0.004286,478.387477,0.003695,53.113110,0.005616,136.911009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013597,1018.307699,0.002513,154.518375,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,1.663136e+05,19176.355372,0.000000,1.505080e+06,4.359710e+06,1.440466e+06,2.224232e+06,94549.652893,7.198724e+05,5.718092e+06,1.996135e+06,0.0,363,11710107.0,1,6766.088492
4,가락동,1,0.007314,0.000000,0.000000,0.000000,0.000000,0.003535,160.575913,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.017309,14.076212,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005607,1278.514656,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,1.635449e+04,2375

In [37]:
refined_Join[refined_Join['인구수'].isna()].shape

(50, 119)

# 2.Data Scaling

# 3.데이터 머신러닝 진행

그럼 이제 학습에 사용하게 될 데이터셋 정제 완료

>>참고: 여기서 지금(8.14) 일시적으로 매출건수랑 매출금액이랑 바뀌었음.. 고려하기

스케일링은 이전 단께에서 완료하고 featur selection은 각 학습 단계에서 각각 따로따로 진행하기

In [38]:
refined_Join

,법정동,성별,연령,DIY자재/용품_매출금액,DIY자재/용품_매출건수,e머니/상품권_매출금액,e머니/상품권_매출건수,o2o서비스_매출금액,o3o서비스_매출건수,가공식품_매출금액,가공식품_매출건수,가방_매출금액,가방_매출건수,가전_매출금액,가전_매출건수,건강식품_매출금액,건강식품_매출건수,기부/후원_매출금액,기부/후원_매출건수,기타_매출금액,기타_매출건수,기타결제_매출금액,기타결제_매출건수,기타교육비_매출금액,기타교육비_매출건수,남성뷰티_매출금액,남성뷰티_매출건수,남성속옷실내복_매출금액,남성속옷실내복_매출건수,남성의류_매출금액,남성의류_매출건수,다이어트_매출금액,다이어트_매출건수,디지털_매출금액,디지털_매출건수,레저_매출금액,레저_매출건수,메이크업_매출금액,메이크업_매출건수,모바일상품_매출금액,...,여행_매출금액,여행_매출건수,육아용품서비스_매출금액,육아용품서비스_매출건수,음료_매출금액,음료_매출건수,인테리어소품_매출금액,인테리어소품_매출건수,임산부용품서비스_매출금액,임산부용품서비스_매출건수,자동차_매출금액,자동차_매출건수,취미/특기_매출금액,취미/특기_매출건수,침구/수예_매출금액,침구/수예_매출건수,침대_매출금액,침대_매출건수,테이블가구_매출금액,테이블가구_매출건수,패션소품_매출금액,패션소품_매출건수,향수_매출금액,향수_매출건수,급여입금,가맹점매출입금,연금입금,총소비금액,총수신금액,예적금금액,신탁금액,수익증권금액,신용대출금액,담보대출금액,주택대출금액,전세자금대출금액,n,법정동코드,기준년월,인구수
0,가락동,1,0.003271,0.000000,0.000000,0.017010,114.829192,0.068859,2368.801471,0.004089,52.129961,0.0,0.0,0.000000,0.000000,0.006870,140.998360,0.0,0.0,0.000000,0.000000,0.002781,43.640523,0.0,0.0,0.0,0.0,0.0,0.0,0.004089,220.395634,0.0,0.0,0.010959,948.481832,0.000000,0.000000,0.0,0.0,0.002126,...,0.012267,427.977130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017010,778.052583,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,1.754613e+04,0.000000,0.000000,8.008194e+05,1.848215e+06,2.789248e+06,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,345,11710107.0,1,6113.950527
1,가락동,1,0.004108,0.002876,147.956210,0.026838,492.951619,0.228400,8160.492626,0.014788,1235.999967,0.0,0.0,0.013693,5443.688873,0.016021,566.270335,0.0,0.0,0.003423,77.705831,0.005751,138.999453,0.0,0.0,0.0,0.0,0.0,0.0,0.002876,131.067347,0.0,0.0,0.024511,2217.025742,0.002328,58.895971,0.0,0.0,0.005203,...,0.026154,2030.238251,0.029714,819.877413,0.011365,359.626562,0.003423,218.377890,0.002328,91.513896,0.003971,64.549144,0.017116,1330.792935,0.006299,421.094224,0.00178,852.612414,0.004519,575.573115,0.005751,750.228158,0.0,0,4.397353e+05,0.000000,0.000000,1.319376e+06,4.218648e+06,4.382826e+06,6.705350e+04,0.000000,1.004127e+06,5.345186e+06,0.000000e+00,0.0,411,11710107.0,1,7302.978353
2,가락동,1,0.005877,0.003086,102.466063,0.015281,623.872023,0.127387,5653.578630,0.017778,609.458752,0.0,0.0,0.009844,750.298348,0.023802,4822.735256,0.0,0.0,0.003673,124.502439,0.009257,74.405902,0.0,0.0,0.0,0.0,0.0,0.0,0.004261,222.930019,0.0,0.0,0.023215,929.059670,0.003086,177.368878,0.0,0.0,0.005583,...,0.015428,1439.851349,0.012783,403.410554,0.014105,445.057513,0.003673,671.833098,0.003086,84.733525,0.002498,46.400398,0.012342,1816.535855,0.009257,369.080355,0.00000,0.000000,0.000000,0.000000,0.004261,188.449217,0.0,0,3.328850e+05,0.000000,0.000000,1.353792e+06,4.582468e+06,2.060297e+06,1.074333e+06,111287.081301,1.588877e+06,3.195215e+06,3.981676e+06,0.0,369,11710107.0,1,6806.019252
3,가락동,1,0.007390,0.002513,409.901083,0.008572,391.508773,0.032663,1529.498914,0.004286,297.993886,0.0,0.0,0.003695,815.622203,0.005616,474.289984,0.0,0.0,0.000000,0.000000,0.065326,409.020072,0.0,0.0,0.0,0.0,0.0,0.0,0.002513,143.152724,0.0,0.0,0.007981,1628.415001,0.001921,100.927146,0.0,0.0,0.000000,...,0.004286,478.387477,0.003695,53.113110,0.005616,136.911009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013597,1018.307699,0.002513,154.518375,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,1.663136e+05,19176.355372,0.000000,1.505080e+06,4.359710e+06,1.440466e+06,2.224232e+06,94549.652893,7.198724e+05,5.718092e+06,1.996135e+06,0.0,363,11710107.0,1,6766.088492
4,가락동,1,0.007314,0.000000,0.000000,0.000000,0.000000,0.003535,160.575913,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.017309,14.076212,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005607,1278.514656,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,1.635449e+04,2375

### (1) 성별예측하기

### (2) 연령예측하기 

### (3) 가구 형태 예측하기// 여기서는 다른 데이터를 사용해야할듯

### (4) 회귀분석
      1. 특정품목 ~~ 다른 품목들간의 관계 우선 3가지 정도 분석해보기
      2. 여행~~ 금융자산정보
      3. 여가~~ 금융자산정보
      4.구매력(총수신금액)~ 구매물품 정보 : 구매품을 분석해서 개인의 구매력 예측가능